# Analysis of key generation data

In [ ]:
 %matplotlib notebook 
import numpy as np
from scipy.stats import describe
from scipy.stats import norm as norm_dist
from scipy.stats.mstats import mquantiles
from math import log, sqrt
import matplotlib.pyplot as plt
from matplotlib import ticker, colors, gridspec
from copy import deepcopy
from utils import plot_hist, moving_average, hw
from binascii import unhexlify
from IPython.display import display, HTML
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import tabulate

## Settings
Enter your input below.

In [ ]:
# File name with output from ECTesterReader or ECTesterStandalone ECDH.
fname = "filename.csv"

# The amount of entries skipped from the beginning of the file, as they are usually outliers.
skip_first = 10

# Whether to plot things in logarithmic scale or not.
log_scale = False

# Whether to trim the time data outside the 1 - 99 percentile range (adjust below). Quite useful.
trim = True

# How much to trim? Either a number in [0,1] signifying a quantile, or an absolute value signifying a threshold
trim_low = 0.01
trim_high = 0.99

# Graphical (matplotlib) style name
style = "ggplot"

# Color map to use, and what color to assign to "bad" values (necessary for log_scale)
color_map = plt.cm.plasma
color_map_bad = "black"

# What function to use to calculate number of histogram bins of time
# one of "sqrt", "sturges", "rice", "scott" and "fd" or a number specifying the number of bins
hist_size = "rice"

## Data processing

In [ ]:
# Setup plot style

plt.style.use(style)

cmap = deepcopy(color_map)
cmap.set_bad(color_map_bad)

# Normalization, linear or log.
if log_scale:
    norm = colors.LogNorm()
else:
    norm = colors.Normalize()

In [ ]:
# Read the header line.

with open(fname, "r") as f:
    header = f.readline()
header_names = header.split(";")
if len(header_names) != 5:
    print("Bad data?")
    exit(1)

# Load the data

hx = lambda x: int(x, 16)
data = np.genfromtxt(fname, delimiter=";", skip_header=1, converters={2: unhexlify, 3: hx, 4: hx},
                         dtype=np.dtype([("index", "u4"), ("time", "u4"), ("pub", "O"), ("priv", "O"), ("secret", "O")]))

time_unit = "ms"

In [ ]:
# Setup the data

# Skip first (outliers?)

data = data[skip_first:]

# If in nanoseconds, scale to microseconds
if header_names[1].endswith("[nano]") and time_unit == "ms":
    time_unit = r"$\mu s$"
    np.floor_divide(data["time"], 1000, out=data["time"])


# Trim times
quant_low_bound = trim_low if 0 <= trim_low <= 1 else 0.01
quant_high_bound = trim_high if 0 <= trim_high <= 1 else 0.95
quantiles = mquantiles(data["time"], prob=(quant_low_bound, 0.25, 0.5, 0.75, quant_high_bound))
if trim:
    low_bound = quantiles[0] if 0 <= trim_low <= 1 else trim_low
    high_bound = quantiles[4] if 0 <= trim_high <= 1 else trim_high
    data_trimmed = data[np.logical_and(data["time"] >= low_bound,
                                       data["time"] <= high_bound)]
    quantiles_trim = mquantiles(data_trimmed["time"], prob=(quant_low_bound, 0.25, 0.5, 0.75, quant_high_bound))
else:
    low_bound = None
    high_bound = None
    data_trimmed = data
    quantiles_trim = quantiles_gen

description = describe(data["time"])
description_trim = describe(data_trimmed["time"])

max_time = description.minmax[1]
min_time = description.minmax[0]
bit_size = len(bin(max(data["priv"]))) - 2
byte_size = (bit_size + 7) // 8

if hist_size == "sqrt":
    hist_size_func = lambda n, xmin, xmax, var, xlower, xupper: int(sqrt(n)) + 1
elif hist_size == "sturges":
    hist_size_func = lambda n, xmin, xmax, var, xlower, xupper: int(log(n, 2)) + 1
elif hist_size == "rice":
    hist_size_func = lambda n, xmin, xmax, var, xlower, xupper: int(2 * n**(1/3))
elif hist_size == "scott":
    hist_size_func = lambda n, xmin, xmax, var, xlower, xupper: (xmax - xmin) // int((3.5 * sqrt(var)) / (n**(1/3)))
elif hist_size == "fd":
    hist_size_func = lambda n, xmin, xmax, var, xlower, xupper: (xmax - xmin) // int(2 * (xupper - xlower) / (n**(1/3)))
else:
    hist_size_func = lambda n, xmin, xmax, var, xlower, xupper: hist_size

hist_size_time = hist_size_func(description.nobs, min_time, max_time, description.variance, quantiles[1], quantiles[3])
hist_size_time_trim = hist_size_func(description_trim.nobs, description_trim.minmax[0], description_trim.minmax[1], description_trim.variance, quantiles_trim[1], quantiles_trim[3])

if hist_size_time < 30:
    hist_size_time = max_time - min_time
if hist_size_time_trim < 30:
    hist_size_time_trim = description_trim.minmax[1] - description_trim.minmax[0]

## Analysis

### Summary

In [ ]:
display("Raw")
desc = [("N", "min, max", "mean", "variance", "skewness", "kurtosis"),
        description]
display(HTML(tabulate.tabulate(desc, tablefmt="html")))
display("Trimmed")
desc = [("N", "min, max", "mean", "variance", "skewness", "kurtosis"),
        description_trim]
display(HTML(tabulate.tabulate(desc, tablefmt="html")))

### Selected quantiles

In [ ]:
tbl = [(quant_low_bound, "0.25", "0.5", "0.75", quant_high_bound),
       list(map(lambda x: "{} {}".format(x, time_unit), quantiles))]
display(HTML(tabulate.tabulate(tbl, tablefmt="html")))

### Info

In [ ]:
display("Bitsize: {}".format(bit_size))
display("Histogram time bins: {}".format(hist_size_time))
display("Histogram time bins(trimmed): {}".format(hist_size_time_trim))

## Plots

### Private key MSB vs time heatmap

In [ ]:
fig_private = plt.figure(figsize=(10.5, 8), dpi=90)
axe_private = fig_private.add_subplot(1, 1, 1)
priv_msb = np.array(list(map(lambda x: x >> (bit_size - 8), data_trimmed["priv"])), dtype=np.dtype("u1"))
max_msb = max(priv_msb)
min_msb = min(priv_msb)
heatmap, xedges, yedges = np.histogram2d(priv_msb, data_trimmed["time"],
                                         bins=[max_msb - min_msb + 1, hist_size_time_trim])
extent = [min_msb, max_msb, yedges[0], yedges[-1]]
im = axe_private.imshow(heatmap.T, extent=extent, aspect="auto", cmap=cmap, origin="low",
                   interpolation="nearest", norm=norm)
axe_private.set_xlabel("private key MSB value")
axe_private.set_ylabel("key agreement time ({})".format(time_unit))
fig_private.colorbar(im, ax=axe_private)

del priv_msb

### Private key Hamming Weight vs time heatmap

In [ ]:
fig_priv_hist = plt.figure(figsize=(10.5, 12), dpi=90)
gs = gridspec.GridSpec(2, 1, height_ratios=[2.5, 1])
axe_priv_hist = fig_priv_hist.add_subplot(gs[0])
axe_priv_hist_hw = fig_priv_hist.add_subplot(gs[1], sharex = axe_priv_hist)
priv_hw = np.array(list(map(hw, data_trimmed["priv"])), dtype=np.dtype("u2"))
h, xe, ye = np.histogram2d(priv_hw, data_trimmed["time"], bins=[max(priv_hw) - min(priv_hw), hist_size_time_trim])
im = axe_priv_hist.imshow(h.T, origin="low", cmap=cmap, aspect="auto", extent=[xe[0], xe[-1], ye[0], ye[-1]], norm=norm)
axe_priv_hist.axvline(x=bit_size//2, alpha=0.7, linestyle="dotted", color="white", label=str(bit_size//2) + " bits")
axe_priv_hist.set_xlabel("private key Hamming weight")
axe_priv_hist.set_ylabel("key agreement time ({})".format(time_unit))
axe_priv_hist.legend(loc="best")

plot_hist(axe_priv_hist_hw, priv_hw, "private key Hamming weight", log_scale, None)

param = norm_dist.fit(priv_hw)
pdf_range = np.arange(min(priv_hw), max(priv_hw))
norm_pdf = norm_dist.pdf(pdf_range, *param[:-2], loc=param[-2], scale=param[-1]) * description_trim.nobs
axe_priv_hist_hw.plot(pdf_range, norm_pdf, label="fitted normal distribution")
axe_priv_hist_hw.legend(loc="best")
fig_priv_hist.colorbar(im, ax=[axe_priv_hist, axe_priv_hist_hw])

display(HTML("<b>Private key Hamming weight fitted with normal distribution:</b>"))
display(HTML(tabulate.tabulate([("Mean", "Variance"), param], tablefmt="html")))

del priv_hw

### Key agreement time histogram

In [ ]:
fig_ka_hist = plt.figure(figsize=(10.5, 8), dpi=90)
axe_hist_full = fig_ka_hist.add_subplot(2, 1, 1)
axe_hist_trim = fig_ka_hist.add_subplot(2, 1, 2)
plot_hist(axe_hist_full, data["time"], "key agreement time ({})".format(time_unit), log_scale, hist_size_time);
plot_hist(axe_hist_trim, data_trimmed["time"], "key agreement time ({})".format(time_unit), log_scale, hist_size_time_trim);

### Moving averages of key agreement time

In [ ]:
fig_avg = plt.figure(figsize=(10.5, 7), dpi=90)
axe_avg = fig_avg.add_subplot(1, 1, 1)
avg_100 = moving_average(data["time"], 100)
avg_1000 = moving_average(data["time"], 1000)
axe_avg.plot(avg_100, label="window = 100")
axe_avg.plot(avg_1000, label="window = 1000")
if low_bound is not None:
    axe_avg.axhline(y=low_bound, alpha=0.7, linestyle="dotted", color="green", label="Low trim bound = {}".format(low_bound))
if high_bound is not None:
    axe_avg.axhline(y=high_bound, alpha=0.7, linestyle="dotted", color="orange", label="Hight trim bound = {}".format(high_bound))
axe_avg.set_ylabel("key agreement time ({})".format(time_unit))
axe_avg.set_xlabel("index")
axe_avg.legend(loc="best")

del avg_100, avg_1000

### Private key MSB and LSB histograms

In [ ]:
fig_priv_hists = plt.figure(figsize=(10.5, 8), dpi=90)
priv_msb = np.array(list(map(lambda x: x >> (bit_size - 8), data["priv"])), dtype=np.dtype("u1"))
priv_lsb = np.array(list(map(lambda x: x & 0xff, data["priv"])), dtype=np.dtype("u1"))
axe_msb_s_hist = fig_priv_hists.add_subplot(2, 1, 1)
axe_lsb_s_hist = fig_priv_hists.add_subplot(2, 1, 2)
msb_h = plot_hist(axe_msb_s_hist, priv_msb, "private key MSB", log_scale, False, False)
lsb_h = plot_hist(axe_lsb_s_hist, priv_lsb, "private key LSB", log_scale, False, False)

del priv_msb, priv_lsb

### Public key coordinate MSB and LSB histograms

In [ ]:
def _split(xy):
    x = int.from_bytes(xy[1:byte_size + 1], byteorder="big")
    y = int.from_bytes(xy[1 + byte_size:], byteorder="big")
    return (x, y)

pub_coords = np.array(list(map(_split, data["pub"])), dtype=np.dtype("O"))
xs = pub_coords[...,0]
ys = pub_coords[...,1]
fig_pub_hists = plt.figure(figsize=(10.5, 14), dpi=90)

def _plot_coord(data, name, offset):
    axe_msb_pub_hist = fig_pub_hists.add_subplot(4, 1, offset)
    axe_lsb_pub_hist = fig_pub_hists.add_subplot(4, 1, offset + 1)
    pub_msb = np.array(list(map(lambda x: x >> (bit_size - 8), data)))
    pub_lsb = np.array(list(map(lambda x: x & 0xff, data)))
    plot_hist(axe_msb_pub_hist, pub_msb, "{} coordinate MSB".format(name), log_scale)
    plot_hist(axe_lsb_pub_hist, pub_lsb, "{} coordinate LSB".format(name), log_scale)
    del pub_msb, pub_lsb

_plot_coord(xs, "X", 1)
_plot_coord(ys, "Y", 3)

del pub_coords, xs, ys

### Private key bit length histogram

In [ ]:
fig_bl = plt.figure(figsize=(10.5, 12), dpi=90)
gs = gridspec.GridSpec(2, 1, height_ratios=[2.5, 1])
axe_bl_heat = fig_bl.add_subplot(gs[0])
axe_bl_hist = fig_bl.add_subplot(gs[1], sharex=axe_bl_heat)
bl_data = np.array(list(map(lambda x: x.bit_length(), data_trimmed["priv"])), dtype=np.dtype("u2"))

h, xe, ye = np.histogram2d(bl_data, data_trimmed["time"], bins=[max(bl_data) - min(bl_data), hist_size_time_trim])
im = axe_bl_heat.imshow(h.T, origin="low", cmap=cmap, aspect="auto", extent=[xe[0], xe[-1], ye[0], ye[-1]], norm=norm)
axe_bl_heat.set_xlabel("private key bit length")
axe_bl_heat.set_ylabel("key agreement time ({})".format(time_unit))

plot_hist(axe_bl_hist, bl_data, "Private key bit length", log_scale, align="right")
fig_bl.colorbar(im, ax=[axe_bl_heat, axe_bl_hist])

del bl_data